In [1]:
from standard_labels import *

In [2]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
def Table2DataFrame(table):
    d1 = { k: table[k][()].flatten() for k in table.keys() if table[k].shape[1]==1}
    d2 = { k+"_"+str(c): table[k][:,c].flatten() for k in table.keys() for c in range(0,table[k].shape[1]) if table[k].shape[1]!=1}
    df = pd.DataFrame( {**d2,**d1} )
    df = df.rename(columns={"event_id_0": "run", "event_id_1": "sub", "event_id_2": "evt"})
    return df

In [4]:
f = h5py.File("NeutrinoML_Open_WithWire_ts331712.h5", "r")
print("\nprinting top-level keys:")
print(f)
print(f.keys())


printing top-level keys:
<HDF5 file "NeutrinoML_Open_WithWire_ts331712.h5" (mode r)>
<KeysViewHDF5 ['edep_table', 'event_table', 'hit_table', 'opflash_table', 'ophit_table', 'pandoraHit_table', 'pandoraPfp_table', 'pandoraPrimary_table', 'particle_table', 'wire_table']>


In [5]:
hits = Table2DataFrame(f["hit_table"])
hits

,run,sub,evt,hit_id,integral,local_plane,local_time,local_wire,rms,tpc
0,6506,128,6401,0,57.365513,0,3570.047363,25.0,4.023290,0
1,6506,128,6401,1,83.823898,0,3566.126465,26.0,3.572362,0
2,6506,128,6401,2,115.995720,0,3561.484375,27.0,3.823352,0
3,6506,128,6401,3,282.163055,0,3557.579102,28.0,3.376822,0
4,6506,128,6401,4,152.136459,0,3555.646729,29.0,3.524922,0
...,...,...,...,...,...,...,...,...,...,...
1711587,6506,137,6898,14488,213.966019,2,6213.527344,3430.0,3.043736,0
1711588,6506,137,6898,14489,138.097916,2,486.424438,3431.0,4.086672,0
1711589,6506,137,6898,14490,78.229744,2,5725.056152,3431.0,3.762090,0
1711590,6506,137,6898,14491,135.898941,2,489.358307,3432.0,4.502100,0


In [6]:
edeps = Table2DataFrame(f["edep_table"])
edeps

,run,sub,evt,energy,energy_fraction,g4_id,hit_id
0,6506,128,6401,1.946514,0.999928,3,211
1,6506,128,6401,0.000141,0.000072,6,211
2,6506,128,6401,0.170546,0.031133,98,215
3,6506,128,6401,0.027469,0.005014,4,215
4,6506,128,6401,0.397784,0.072614,3,215
...,...,...,...,...,...,...,...
553444,6506,137,6898,0.792252,1.000000,2614,13093
553445,6506,137,6898,0.794417,0.944855,2615,13332
553446,6506,137,6898,0.046365,0.055145,2625,13332
553447,6506,137,6898,0.694303,0.779218,2615,13333


In [7]:
edeps = edeps.sort_values(by=['energy_fraction'], ascending=False, kind='mergesort').drop_duplicates(["run","sub","evt","hit_id"])
edeps

,run,sub,evt,energy,energy_fraction,g4_id,hit_id
26,6506,128,6401,5.399584,1.000000,2,223
27,6506,128,6401,0.001778,1.000000,2,224
28,6506,128,6401,0.011668,1.000000,685,230
33,6506,128,6401,5.728998,1.000000,2,234
38,6506,128,6401,0.512232,1.000000,1,241
...,...,...,...,...,...,...,...
161211,6506,131,6569,1.515597,0.095550,3506,6063
161128,6506,131,6569,1.379928,0.094543,918,6053
274558,6506,133,6676,1.980549,0.094249,1254,10710
167448,6506,131,6569,0.728176,0.090718,3422,18382


In [8]:
hits = hits.merge(edeps, on=["run","sub","evt","hit_id"], how="left")
hits['g4_id'] = hits['g4_id'].fillna(-1)
hits = hits.fillna(0)
hits

,run,sub,evt,hit_id,integral,local_plane,local_time,local_wire,rms,tpc,energy,energy_fraction,g4_id
0,6506,128,6401,0,57.365513,0,3570.047363,25.0,4.023290,0,0.0,0.0,-1.0
1,6506,128,6401,1,83.823898,0,3566.126465,26.0,3.572362,0,0.0,0.0,-1.0
2,6506,128,6401,2,115.995720,0,3561.484375,27.0,3.823352,0,0.0,0.0,-1.0
3,6506,128,6401,3,282.163055,0,3557.579102,28.0,3.376822,0,0.0,0.0,-1.0
4,6506,128,6401,4,152.136459,0,3555.646729,29.0,3.524922,0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711587,6506,137,6898,14488,213.966019,2,6213.527344,3430.0,3.043736,0,0.0,0.0,-1.0
1711588,6506,137,6898,14489,138.097916,2,486.424438,3431.0,4.086672,0,0.0,0.0,-1.0
1711589,6506,137,6898,14490,78.229744,2,5725.056152,3431.0,3.762090,0,0.0,0.0,-1.0
1711590,6506,137,6898,14491,135.898941,2,489.358307,3432.0,4.502100,0,0.0,0.0,-1.0


In [9]:
parts = Table2DataFrame(f["particle_table"])
parts

,end_position_0,end_position_1,end_position_2,end_position_corr_0,end_position_corr_1,end_position_corr_2,end_wire_pos_0,end_wire_pos_1,end_wire_pos_2,run,...,start_wire_pos_1,start_wire_pos_2,end_process,end_wire_time,g4_id,momentum,parent_id,start_process,start_wire_time,type
0,181.163284,80.766945,318.862488,183.907227,76.948967,318.998901,648,1086,1062,6506,...,1114,967,b'FastScintillation',4150.073730,1,0.652232,0,b'primary',4185.353027,11
1,195.016190,117.518143,314.333038,196.981613,111.647591,314.405579,540,1179,1047,6506,...,1114,967,b'FastScintillation',4388.237793,2,0.664269,0,b'primary',4185.353027,2212
2,140.170273,201.012360,385.474640,141.176788,201.012360,385.474640,400,1555,1284,6506,...,1114,967,b'Decay',3371.691406,3,0.346863,0,b'primary',4185.353027,211
3,183.822723,108.486115,290.700836,185.672134,103.563652,290.806274,524,1116,969,6506,...,1116,969,b'FastScintillation',4182.223145,4,0.000619,3,b'hIoni',4182.387695,11
4,183.556915,109.110977,291.388763,185.407227,104.191833,291.492340,523,1119,971,6506,...,1119,971,b'FastScintillation',4177.397461,5,0.000729,3,b'hIoni',4177.724609,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76823,139.253082,-52.975441,388.751190,143.607071,-51.621639,388.855621,1135,832,1295,6506,...,831,1295,b'FastScintillation',3415.961670,5881,0.000827,5879,b'eIoni',3414.860107,11
76824,203.246033,3.998529,412.545868,206.968124,2.187929,412.772522,1020,1027,1375,6506,...,1027,1376,b'FastScintillation',4570.153320,5886,0.002731,5733,b'compt',4560.353027,11
76825,212.245697,11.581883,420.274384,215.737411,10.054076,420.694031,1010,1063,1401,6506,...,1063,1402,b'FastScintillation',4729.895508,5887,0.001030,5733,b'compt',4727.338379,11
76826,212.813980,12.945192,422.494324,216.291779,11.375595,422.923492,1010,1070,1409,6506,...,1070,1409,b'FastScintillation',4739.994141,5888,0.001417,5733,b'compt',4737.541504,11


## Produce scatter plots for specific event

In [81]:
ievt = 7 #2

In [82]:
print("\nnow i can pull out a single entry in the event table:")
evts = f["event_table/event_id"]
evt = evts[ievt]
print(evt)


now i can pull out a single entry in the event table:
[6506  128 6448]


In [83]:
print("\nand i can pull out particle info for this event specifically:")
evtmask = (parts["run"]==evt[0])&(parts["sub"]==evt[1])&(parts["evt"]==evt[2])
evtparts = parts[evtmask]


and i can pull out particle info for this event specifically:


In [84]:
print("\nand i can pull out hit info for this event specifically:")
evtmask = (hits["run"]==evt[0])&(hits["sub"]==evt[1])&(hits["evt"]==evt[2])
evthits = hits[evtmask]


and i can pull out hit info for this event specifically:


In [85]:
evtpartlabls = panoptic_label(evtparts)

In [86]:
evtpartlabls

,g4_id,parent_id,type,start_process,end_process,momentum,semantic_label,instance_label
0,1,0,11,b'primary',b'FastScintillation',1.063347,4,0
1,24,1,11,b'eIoni',b'FastScintillation',0.015009,4,0
2,128,24,22,b'eBrem',b'phot',0.004459,4,0
3,133,128,11,b'compt',b'FastScintillation',0.004190,4,0
4,141,133,11,b'eIoni',b'FastScintillation',0.000641,4,0
...,...,...,...,...,...,...,...,...
944,8,0,2112,b'primary',b'neutronInelastic',0.072271,7,-1
945,10,8,22,b'neutronInelastic',b'phot',0.001994,7,-1
946,678,10,11,b'compt',b'FastScintillation',0.001391,7,-1
947,759,678,22,b'eBrem',b'phot',0.000011,7,-1


In [87]:
print("\nand i can pull out particle info for this event specifically:")
evtmask = (edeps["run"]==evt[0])&(edeps["sub"]==evt[1])&(edeps["evt"]==evt[2])
evtedeps = edeps[evtmask]


and i can pull out particle info for this event specifically:


In [88]:
evthits = evthits.merge(evtpartlabls, on="g4_id", how="left")

## Plot semantic label of hits

In [89]:
evthits["semantic_label_name"] = 'cosmic'
for l in label:
    #print(l.name,l.value)
    evthits.loc[evthits["semantic_label"]==l.value, "semantic_label_name"] = l.name

color_dict = {"pion" : "yellow",
      "muon" : "green",
      "kaon" : "black",
      "hadron" : "blue",
      "shower" : "red",
      "michel" : "purple",
      "delta" : "pink",
      "diffuse" : "orange",
      "invisible" : "white",
      "HIP": "blue",
      "MIP": "green",
      "cosmic": "gray"}

fig = px.scatter(evthits, x="local_wire", y="local_time", color="semantic_label_name", color_discrete_map=color_dict, facet_col="local_plane", 
                 labels={
                     "wire": "Wire",
                     "time": "Time Tick",
                     "semantic_label_name": "Class"
                 },
                 title="semantic_label plot")
fig.show()


## Plot instance label of hits

In [90]:
evthits.loc[evthits["g4_id"]<0,"instance_label"] = -1
evthits["instance_label_str"] = evthits["instance_label"].astype('string')

fig = px.scatter(evthits, x="local_wire", y="local_time", color="instance_label_str", facet_col="local_plane", 
                 labels={
                     "wire": "Wire",
                     "time": "Time Tick",
                     "instance_label_str": "Instance"
                 },
                 title="instance_label plot")
fig.show()
